In [1]:
import pandas as pd
import ast

In [2]:
def replace_token(x, representation_style):
    # x: pandas row
    # 문자열(x['sentence']) -> 리스트로 변환 후 삭제하고 해당 인덱스에 삽입
    x_sentence = x['sentence']

    # ['baseline', 'klue', 'matching_the_blank', 'punct', 'typed_marker', 'typed_punct_marker']

    if representation_style == 'klue':
        entity_markers = {'subj_s' : '<subj>', 'subj_e' : '</subj>', 'obj_s' : '<obj>', 'obj_e' : '</obj>'}
    elif representation_style == 'matching_the_blank':
        entity_markers = {'subj_s' : '[E1]', 'subj_e' : '</E1>', 'obj_s' : '<E2>', 'obj_e' : '</E2>'}
    elif representation_style == 'punct':
        entity_markers = {'subj_s' : '@', 'subj_e' : '@', 'obj_s' : '#', 'obj_e' : '#'}
        

    subj_s = x['subject_entity']['start_idx']
    subj_e = x['subject_entity']['end_idx']
    obj_s = x['object_entity']['start_idx']
    obj_e = x['object_entity']['end_idx']

    subj_type = x['subject_entity']['type']
    obj_type = x['object_entity']['type']

    subj_word = x['subject_entity']['word']
    obj_word = x['object_entity']['word']

    
    # entity type 포함
    if representation_style == 'typed_marker':
        # [CLS]〈Something〉는 <o:PER>조지 해리슨</o:PER>이 쓰고 <s:ORG>비틀즈</s:ORG>가 1969년 앨범 《Abbey Road》에 담은 노래다.[SEP]
        if subj_s < obj_s: # subj 가 먼저 나올 때
            tmp = f'{x_sentence[:subj_s]}<s:{subj_type}>{subj_word}</s:{subj_type}>{x_sentence[subj_e+1:obj_s]}<o:{obj_type}>{obj_word}</o:{obj_type}>{x_sentence[obj_e+1:]}'
        
        elif subj_s > obj_s: # obj 가 먼저 나올 때
            tmp = f'{x_sentence[:obj_s]}<o:{obj_type}>{obj_word}</o:{obj_type}>{x_sentence[obj_e+1:subj_s]}<s:{subj_type}>{subj_word}</s:{subj_type}>{x_sentence[subj_e+1:]}'

        else:
            raise ValueError("subj-obj overlapped")
    

    elif representation_style == 'typed_punct_marker':
        # [CLS]〈Something〉는 #^PER^조지 해리슨#이 쓰고 @*ORG*비틀즈@가 1969년 앨범 《Abbey Road》에 담은 노래다.[SEP]
        if subj_s < obj_s: # subj 가 먼저 나올 때
            tmp = f'{x_sentence[:subj_s]}@*{subj_type}*{subj_word}@{x_sentence[subj_e+1:obj_s]}#^{obj_type}^{obj_word}#{x_sentence[obj_e+1:]}'
        
        elif subj_s > obj_s: # obj 가 먼저 나올 때
            tmp = f'{x_sentence[:obj_s]}#^{obj_type}^{obj_word}#{x_sentence[obj_e+1:subj_s]}@*{subj_type}*{subj_word}@{x_sentence[subj_e+1:]}'
        else:
            raise ValueError("subj-obj overlapped")


    # entity type 불포함
    else:
        # f-string 말고 그냥 더하기로..
        if subj_s < obj_s: # subj 가 먼저 나올 때
            tmp = x_sentence[:subj_s] + entity_markers['subj_s'] + subj_word + entity_markers['subj_e'] + x_sentence[subj_e+1:obj_s] + entity_markers['obj_s'] + obj_word + entity_markers['obj_e'] + x_sentence[obj_e+1:]

        elif subj_s > obj_s: # obj 가 먼저 나올 때
            tmp = x_sentence[:obj_s] + entity_markers['obj_s'] + obj_word + entity_markers['obj_e'] + x_sentence[obj_e+1:subj_s] + entity_markers['subj_s'] + subj_word + entity_markers['subj_e'] + x_sentence[subj_e+1:]

        else:
            raise ValueError("subj-obj overlapped")

    return tmp

In [3]:
pd_dataset = pd.read_csv('/data/ephemeral/level2-klue-nlp-06/dataset/train/train.csv')
pd_dataset.head()

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia


In [4]:
pd_dataset.loc[0, 'subject_entity']

"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}"

In [5]:
pd_dataset.loc[0, 'object_entity']

"{'word': '조지 해리슨', 'start_idx': 13, 'end_idx': 18, 'type': 'PER'}"

In [6]:
pd_dataset['subject_entity'] = pd_dataset['subject_entity'].apply(lambda x: ast.literal_eval(x))
pd_dataset['object_entity'] = pd_dataset['object_entity'].apply(lambda x: ast.literal_eval(x))
pd_dataset

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


In [7]:
replace_token(pd_dataset.loc[0], 'klue')

'〈Something〉는 <obj>조지 해리슨</obj>이 쓰고 <subj>비틀즈</subj>가 1969년 앨범 《Abbey Road》에 담은 노래다.'

In [8]:
pd_dataset['sentence'] = pd_dataset.apply(lambda x : replace_token(x, 'klue'), axis=1)
pd_dataset

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 <obj>조지 해리슨</obj>이 쓰고 <subj>비틀즈</...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·<obj>대안신당</obj>·<subj>민주평화당</sub...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 <subj>광주FC</subj>는 지난 26일...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)<subj>아성다이소</subj>(대표 <obj>박정부</o...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,<obj>1967</obj>년 프로 야구 드래프트 1순위로 <subj>요미우리 자이...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 <subj>최시형</subj>, 서병학, <obj>손병희</obj> 직...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,<subj>완도군</subj>(군수 <obj>신우철</obj>)이 국토교통부에서 실...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, <subj>JTBC</subj> 회장을 지낸 이후 <obj>중앙홀딩스</...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


----

## 제대로 detokenize 되는지 확인!

In [9]:
from transformers import AutoTokenizer, AutoConfig

/opt/conda/envs/klue_re/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large', max_length=256)

In [11]:
print("### special tokens in current tokenizer : ", tokenizer.special_tokens_map)

### special tokens in current tokenizer :  {'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}


In [12]:
special_tokens_dict = {'additional_special_tokens': ['<subj>', '</subj>', '<obj>', '</obj>']}

In [13]:
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print('We have added', num_added_toks, 'tokens.')

print("### special tokens in current tokenizer : ", tokenizer.special_tokens_map)

We have added 4 tokens.
### special tokens in current tokenizer :  {'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['<subj>', '</obj>', '<obj>', '</subj>']}


In [14]:
pd_dataset.loc[0, 'sentence']

'〈Something〉는 <obj>조지 해리슨</obj>이 쓰고 <subj>비틀즈</subj>가 1969년 앨범 《Abbey Road》에 담은 노래다.'

In [15]:
vocab_id = tokenizer.encode(pd_dataset.loc[0, 'sentence'])
vocab_id

[0,
 168,
 30985,
 14451,
 7088,
 4586,
 169,
 793,
 32002,
 8373,
 14113,
 2234,
 32001,
 1504,
 1363,
 2088,
 32000,
 29830,
 32003,
 543,
 14879,
 2440,
 6711,
 170,
 21406,
 26713,
 2076,
 25145,
 5749,
 171,
 1421,
 818,
 2073,
 4388,
 2062,
 18,
 2]

In [16]:
# 예시문장 decode
tokenizer.decode(vocab_id)

'[CLS] 〈 Something 〉 는 <obj> 조지 해리슨 </obj> 이 쓰고 <subj> 비틀즈 </subj> 가 1969년 앨범 《 Abbey Road 》 에 담은 노래다. [SEP]'

In [17]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

In [18]:
model_config =  AutoConfig.from_pretrained('klue/roberta-large')
model_config.num_labels = 30

model = AutoModelForSequenceClassification.from_pretrained('klue/roberta-large', config=model_config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
print(model)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [20]:
# word embedding 32000 -> 32004
model.resize_token_embeddings(len(tokenizer))

Embedding(32004, 1024)

## unique entity type

In [29]:
total_type_list = []
s_type_list = pd_dataset['subject_entity'].apply(lambda x : x['type'])
o_type_list = pd_dataset['object_entity'].apply(lambda x : x['type'])

s_type_list = s_type_list.unique(); o_type_list = o_type_list.unique()

In [31]:
s_type_list

array(['ORG', 'PER'], dtype=object)

In [32]:
o_type_list

array(['PER', 'ORG', 'DAT', 'LOC', 'POH', 'NOH'], dtype=object)

In [38]:
total_type_list = list('<s:' + s_type_list + '>')
total_type_list.extend('</s:' + s_type_list + '>')
total_type_list.extend('<o:' + o_type_list + '>')
total_type_list.extend('</o:' + o_type_list + '>')

print('### total type list :', total_type_list)
special_tokens_dict = {'additional_special_tokens': total_type_list}

### total type list : ['<s:ORG>', '<s:PER>', '</s:ORG>', '</s:PER>', '<o:PER>', '<o:ORG>', '<o:DAT>', '<o:LOC>', '<o:POH>', '<o:NOH>', '</o:PER>', '</o:ORG>', '</o:DAT>', '</o:LOC>', '</o:POH>', '</o:NOH>']


## tokenizer shape test

In [5]:
from transformers import AutoTokenizer, AutoConfig

tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large', max_length=256)

sequence_a = "이것은 짧은 시퀀스 입니다."

padded_sequences = tokenizer(sequence_a, 
            padding=True, return_tensors="pt",
            truncation=True,
            max_length=256)

print(padded_sequences)

{'input_ids': tensor([[   0, 3982, 2073, 1599, 2073,    3, 3714,   18,    2]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


# Dataset shape

In [ ]:
import torch

In [ ]:
class RE_Dataset(torch.utils.data.Dataset):
  """ Dataset 구성을 위한 class."""
  def __init__(self, pair_dataset, labels):
    self.pair_dataset = pair_dataset
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

# pooling test

In [1]:
import torch

/opt/conda/envs/klue_re/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pooled_output = torch.zeros(torch.Size([16, 1024]))

In [5]:
pooled_output

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [6]:
ss_emb = torch.ones(torch.Size([16, 1024]))

In [7]:
ss_emb

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [9]:
os_emb = torch.full(torch.Size([16, 1024]), 5)
os_emb

tensor([[5, 5, 5,  ..., 5, 5, 5],
        [5, 5, 5,  ..., 5, 5, 5],
        [5, 5, 5,  ..., 5, 5, 5],
        ...,
        [5, 5, 5,  ..., 5, 5, 5],
        [5, 5, 5,  ..., 5, 5, 5],
        [5, 5, 5,  ..., 5, 5, 5]])

In [26]:
torch.cat([pooled_output, ss_emb, os_emb], dim=-1)

tensor([[0., 0., 0.,  ..., 5., 5., 5.],
        [0., 0., 0.,  ..., 5., 5., 5.],
        [0., 0., 0.,  ..., 5., 5., 5.],
        ...,
        [0., 0., 0.,  ..., 5., 5., 5.],
        [0., 0., 0.,  ..., 5., 5., 5.],
        [0., 0., 0.,  ..., 5., 5., 5.]])

In [29]:
torch.mean(torch.cat([pooled_output, ss_emb, os_emb], dim=-1), dim=-1, keepdim=True).shape

torch.Size([16, 1])

In [ ]:

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [24]:
# 배치 사이즈는 유지하면서 얘네를 연산하기
# 16, 3072 -> 16, 1024
torch.cat([pooled_output, ss_emb, os_emb], dim=-1).shape

torch.Size([16, 3072])

In [23]:
torch.mean(torch.cat([pooled_output, ss_emb, os_emb], dim=-1), dim=0)

tensor([0., 0., 0.,  ..., 5., 5., 5.])

In [21]:
torch.mean(torch.cat([pooled_output, ss_emb, os_emb], dim=-1), dim=0).shape

torch.Size([3072])

In [18]:
torch.cat([pooled_output, ss_emb, os_emb], dim=-1).shape

torch.Size([16, 3072])

In [11]:
h

tensor([0., 0., 0.,  ..., 5., 5., 5.])

h = torch.mean(torch.cat([pooled_output, ss_emb, os_emb], dim=-1), dim=0)